In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/25 18:08:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/25 18:08:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/25 18:08:54 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
data = spark.read.parquet('./data/parquet')

In [4]:
data.show()

+-----+--------------------+
|  dia|              tareas|
+-----+--------------------+
|lunes|[hacer la tarea, ...|
+-----+--------------------+



In [5]:
data.printSchema()

root
 |-- dia: string (nullable = true)
 |-- tareas: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [6]:
from pyspark.sql.functions import col, size, sort_array, array_contains

In [11]:
data.select(
    size(col('tareas')).alias('tamaño'),
    sort_array(col('tareas')).alias('arreglo_ordenado'),
    array_contains(col('tareas'), 'buscar agua').alias('buscar_agua')
).show(truncate=False)

+------+--------------------------------------------+-----------+
|tamaño|arreglo_ordenado                            |buscar_agua|
+------+--------------------------------------------+-----------+
|3     |[buscar agua, hacer la tarea, lavar el auto]|true       |
+------+--------------------------------------------+-----------+



In [12]:
from pyspark.sql.functions import explode

In [14]:
data.select(
    col('dia'),
    explode(col('tareas'))
).show()

+-----+--------------+
|  dia|           col|
+-----+--------------+
|lunes|hacer la tarea|
|lunes|   buscar agua|
|lunes| lavar el auto|
+-----+--------------+



In [15]:
df2 = spark.read.parquet('./data/JSON')

In [17]:
df2.show(truncate=False)

+---------------------------------------------------------------------------+
|tareas_str                                                                 |
+---------------------------------------------------------------------------+
|{"dia": "lunes","tareas": ["hacer la tarea","buscar agua","lavar el auto"]}|
+---------------------------------------------------------------------------+



In [18]:
df2.printSchema()

root
 |-- tareas_str: string (nullable = true)



In [19]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

In [20]:
schema_json = StructType(
    [
        StructField('dia', StringType(), True),
        StructField('tareas', ArrayType(StringType()), True)
    ]
)

In [21]:
from pyspark.sql.functions import from_json, to_json

In [22]:
df3 = df2.select(
    from_json(col('tareas_str'), schema_json).alias('tareas_por_hacer')
)

In [23]:
df3.printSchema()

root
 |-- tareas_por_hacer: struct (nullable = true)
 |    |-- dia: string (nullable = true)
 |    |-- tareas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)



In [25]:
df3.select(
    col('tareas_por_hacer').getItem('dia'),
    col('tareas_por_hacer').getItem('tareas'),
    col('tareas_por_hacer').getItem('tareas')[0]
).show()

+--------------------+-----------------------+--------------------------+
|tareas_por_hacer.dia|tareas_por_hacer.tareas|tareas_por_hacer.tareas[0]|
+--------------------+-----------------------+--------------------------+
|               lunes|   [hacer la tarea, ...|            hacer la tarea|
+--------------------+-----------------------+--------------------------+



In [26]:
df3.select(
    to_json(col('tareas_por_hacer'))
).show(truncate=False)

+-------------------------------------------------------------------------+
|to_json(tareas_por_hacer)                                                |
+-------------------------------------------------------------------------+
|{"dia":"lunes","tareas":["hacer la tarea","buscar agua","lavar el auto"]}|
+-------------------------------------------------------------------------+



25/04/25 23:22:03 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 496929 ms exceeds timeout 120000 ms
25/04/25 23:22:03 WARN SparkContext: Killing executors is not supported by current scheduler.
25/04/25 23:22:10 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B